In [1]:
import numpy as np
from tvm import relay
import tvm
from tvm.contrib import graph_runtime

myrelay = """
v0.0.4

def @myfun(%acc : Tensor[(2, 2), float32],
           %x   : Tensor[(2, 2), float32],
           %n   : int32) -> Tensor[(2, 2), float32] {
  if (%n > 0) {
    @myfun(%acc + %x, %x, %n - 1)
  } else {
    %acc
  }
}
"""

my_relay_module = relay.fromtext(myrelay)
print(my_relay_module.astext())

v0.0.4
def @myfun(%acc: Tensor[(2, 2), float32], %x: Tensor[(2, 2), float32], %n: int32) -> Tensor[(2, 2), float32] {
  %0 = greater(%n, 0 /* ty=int32 */) /* ty=bool */;
  if (%0) {
    %1 = add(%acc, %x) /* ty=Tensor[(2, 2), float32] */;
    %2 = subtract(%n, 1 /* ty=int32 */) /* ty=int32 */;
    @myfun(%1, %x, %2) /* ty=Tensor[(2, 2), float32] */
  } else {
    %acc
  }
}



In [2]:
ctx = tvm.cpu()
opt_level = 3
target = tvm.target.create('llvm')
with relay.build_config(opt_level=opt_level):
    executor = relay.build_module.create_executor(
        'vm', my_relay_module, ctx, target)

myfun_reified = executor.evaluate(my_relay_module.get_global_var('myfun'))
data = np.random.uniform(1, 2, size=(2, 2)).astype('float32')
acc = np.zeros(shape=(2, 2), dtype=np.float32)

out = myfun_reified(acc, data, 10).asnumpy()
print(out)

TVMError: Traceback (most recent call last):
  [bt] (8) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::vm::VMCompiler::OptimizeModule(tvm::relay::Module const&, tvm::Map<tvm::Integer, tvm::Target, void, void> const&)+0xa3a) [0x7ff244d6159a]
  [bt] (7) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::transform::Pass::operator()(tvm::relay::Module const&) const+0x57) [0x7ff244d048c7]
  [bt] (6) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::transform::SequentialNode::operator()(tvm::relay::Module const&, tvm::relay::transform::PassContext const&) const+0x490) [0x7ff244c9dc80]
  [bt] (5) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::transform::ModulePassNode::operator()(tvm::relay::Module const&, tvm::relay::transform::PassContext const&) const+0x100) [0x7ff244c9e2a0]
  [bt] (4) /home/ubuntu/tvm/build/libtvm.so(+0x11eb232) [0x7ff244d77232]
  [bt] (3) /home/ubuntu/tvm/build/libtvm.so(tvm::relay::vm::RemoveUnusedFunctions(tvm::relay::Module const&, tvm::Array<tvm::Expr, void>)+0x247) [0x7ff244d76847